In [ ]:
import glob, os
from joblib import Parallel, delayed
from astroquery.mast import Observations

proposal_id = 1727
instrument_name = "MIRI/IMAGE"
filters = "F770W"
directory = f"./F770W/"


# Get observations for the given query
obs_list = Observations.query_criteria(proposal_id=proposal_id,
                                        instrument_name=instrument_name,
                                        filters=filters)
obs_list_pd = obs_list.to_pandas().sort_values(by='obs_id').reset_index(drop=True)


total_size = 0
#                 change to ":" retrieve ALL the data
#                            |
#                            V
for name in obs_list_pd.iloc[:]['obs_id']:
    # retrieve the data products for the given observation
    print(f"Gathering data products for: {name}", end=' ')

    mask = (obs_list['obs_id'] == name)
    data_products = Observations.get_product_list(obs_list[mask])
    
    # filter the data products for the uncalibrated images
    filtered_prod = Observations.filter_products(data_products, 
                                                productType="SCIENCE",
                                                extension="fits",
                                                calib_level=[1], 
                                                productSubGroupDescription="UNCAL")
    size = sum(filtered_prod['size'])
    total_size += size
    print(f"...retrieving {len(filtered_prod)} files ({size/1e9:.2f} GB)")

    # download the data products
    manifest = Observations.download_products(filtered_prod, curl_flag=False)

print("Finish fetching data products")
print(f"Total size: {total_size/1e9:.2f} GB")

manifest_list = glob.glob("mastDownload*")
print(f"Total files: {len(manifest_list)}")

def download_data_products(manifest):
    print(f"Downloading: {manifest}")
    os.system(f"chmod +x {manifest}")
    os.system(f"./{manifest}")

Parallel(n_jobs=16)(delayed(download_data_products)(manifest) for manifest in manifest_list)

if not os.path.exists(directory):
    os.system(f"mkdir {directory}")

for manifest in manifest_list:
    os.system(f"mv {manifest}/JWST/* {directory}")
    os.system(f"rm -R {manifest}")
    os.system(f"rm ./{manifest}")

In [49]:
import glob, os
from joblib import Parallel, delayed
from astroquery.mast import Observations

proposal_id = 1727
instrument_name = "MIRI/IMAGE"
filters = "F770W"
directory = f"./F770W/"

a = []
obs = []
for _dir in sorted(glob.glob("/mnt/C/JWST/COSMOS/MIRI/F770W/jw*mirimage")):
    file = f"{_dir}/{_dir.split('/')[-1]}_uncal.fits"
    if not os.path.exists(file):
        a.append(_dir.split("/")[-1])
        obs.append(_dir.split("/")[-1].split("_")[0][-6:-3])
obs =list(set(obs))

# Get observations for the given query
obs_list = Observations.query_criteria(proposal_id=proposal_id,
                                        instrument_name=instrument_name,
                                        filters=filters)
obs_list_pd = obs_list.to_pandas().sort_values(by='obs_id').reset_index(drop=True)

total_size = 0
for name in obs_list_pd.iloc[:]['obs_id']:
    if name.split("_")[0][-3:] in obs:
        # retrieve the data products for the given observation
        print(f"Gathering data products for: {name}", end=' ')

        mask = (obs_list['obs_id'] == name)
        data_products = Observations.get_product_list(obs_list[mask])
        
        # filter the data products for the uncalibrated images
        filtered_prod = Observations.filter_products(data_products, 
                                                    productType="SCIENCE",
                                                    extension="fits",
                                                    calib_level=[1], 
                                                    productSubGroupDescription="UNCAL")
        
        size = sum(filtered_prod['size'])
        total_size += size
        print(f"...retrieving {len(filtered_prod)} files ({size/1e9:.2f} GB)")

        # download the data products
        manifest = Observations.download_products(filtered_prod, curl_flag=False)

    else:
        pass
    
print("Finish fetching data products")
print(f"Total size: {total_size/1e9:.2f} GB")

manifest_list = glob.glob("mastDownload*")
print(f"Total files: {len(manifest_list)}")

def download_data_products(manifest):
    print(f"Downloading: {manifest}")
    os.system(f"chmod +x {manifest}")
    os.system(f"./{manifest}")

Parallel(n_jobs=16)(delayed(download_data_products)(manifest) for manifest in manifest_list)

Gathering data products for: jw01727-o045_t027_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o046_t028_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o047_t035_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o048_t036_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o078_t006_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o079_t013_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o080_t014_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o081_t021_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o082_t022_miri_f770w ...retrieving 4 files (0.95 GB)
Gathering data products for: jw01727-o083_t029_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering data products for: jw01727-o084_t030_miri_f770w ...retrieving 8 files (1.90 GB)
Gathering 

sh: 1: ./mastDownload: Permission denied


[None]

In [46]:
print(obs_list_pd.columns)


Index(['intentType', 'obs_collection', 'provenance_name', 'instrument_name',
       'project', 'filters', 'wavelength_region', 'target_name',
       'target_classification', 'obs_id', 's_ra', 's_dec', 'dataproduct_type',
       'proposal_pi', 'calib_level', 't_min', 't_max', 't_exptime', 'em_min',
       'em_max', 'obs_title', 't_obs_release', 'proposal_id', 'proposal_type',
       'sequence_number', 's_region', 'jpegURL', 'dataURL', 'dataRights',
       'mtFlag', 'srcDen', 'obsid', 'objID'],
      dtype='object')


'009'